<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/Seq2Seq_Chatbot_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 전처리
- 특수 문자 제거
- 특수 토큰 추가 MAX_LENGTH인 25에 맞춰서 시작 토큰과 끝 토큰을 추가 -> 이후 field에서 추가해서 사용
- decoder의 입력에 따른 예측값에 해당하는 dataset을 마련

In [ ]:
import pandas as pd

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [ ]:
ls

ChatbotData.csv  Mecab-ko-for-Google-Colab/


In [ ]:
df=pd.read_csv('ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 특수 문자 제거

In [ ]:
df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df.head()

<ipython-input-90-21d281052103>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
<ipython-input-90-21d281052103>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")


,Q,A,label
0,12시 땡,하루가 또 가네요,0
1,1지망 학교 떨어졌어,위로해 드립니다,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,0
4,PPL 심하네,눈살이 찌푸려지죠,0


In [ ]:
def decoder_input(sent,Max_len):
  sent=sent.split()
  if len(sent)>Max_len-2: # Max_length=25
    sent=sent[:Max_len-2]
    sent.append('<eos>')
    sent[:0]=['<sos>']
  else:
    sent.append('<eos>')
    sent[:0]=['<sos>']
  
  return " ".join(sent)

In [ ]:
def decoder_output(sent):
  sent=sent.split()
  sent.pop(0)
  return " ".join(sent)

In [ ]:
# 알고보니 torchtext의 필드정의에서 <sos>, <eos> token 추가를 지원했다. => 이럴 경우 굳이 미리 추가해놓고 준배해줄 필요가 없다.
df['label']=df['A']
temp=pd.Series(df['Q'].tolist()+df['A'].tolist()).astype(str)
temp.to_csv('vocab_word.csv',index=False)
df.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


In [ ]:
df.to_csv('2_Fine_ChatbotData.csv',index=False)

In [ ]:
decoder_input('안녕하세요 저는 사람입니다',4)

'<sos> 안녕하세요 저는 <eos>'

In [ ]:
decoder_output('<sos> 안녕하세요 저는 <eos>')

'안녕하세요 저는 <eos>'

In [ ]:
df['A']=df['A'].apply(lambda x : decoder_input(x,25))
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,0
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,0
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,0


In [ ]:
df['label']=df['A'].apply(decoder_output)
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,하루가 또 가네요 <eos>
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,위로해 드립니다 <eos>
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,눈살이 찌푸려지죠 <eos>


In [ ]:
df.to_csv('Fine_ChatbotData.csv',index=False)

# 데이터 불러오기 및 dataset,dataLoader 생성

In [12]:
import pandas as pd

In [13]:
ls

drive/  sample_data/


In [14]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [15]:
ls

2_Fine_ChatbotData.csv  ChatbotData.csv       Mecab-ko-for-Google-Colab/
best_weight/            Fine_ChatbotData.csv  vocab_word.csv


In [16]:
vocab_word=pd.read_csv('vocab_word.csv')
vocab_word.head()

,0
0,12시 땡
1,1지망 학교 떨어졌어
2,3박4일 놀러가고 싶다
3,3박4일 정도 놀러가고 싶다
4,PPL 심하네


In [17]:
train_data=pd.read_csv('2_Fine_ChatbotData.csv')
train_data.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


tokenizer = 공백기준분리 str.split() 사용 예정


In [18]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256

### 필드 정의 및 Tabulardataset 생성

In [19]:
pip install torchtext==0.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from torchtext import data

In [21]:
# vocab 전용
V=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             init_token='<sos>',
             eos_token='<eos>',
             fix_length=MAX_LEN)

In [22]:
Q=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)

A=data.Field(sequential=True,
             use_vocab=True,
             init_token='<sos>',
             eos_token='<eos>',
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)
LABEL=data.Field(sequential=True,
                 use_vocab=True,
                 eos_token='<eos>',
                 tokenize=str.split,
                 batch_first=True,
                 fix_length=MAX_LEN)

In [23]:
from torchtext.data import TabularDataset

In [24]:
train_data=TabularDataset('2_Fine_ChatbotData.csv',format='csv',fields=[('question',Q),('answer',A),('label',LABEL)],skip_header=True)
vars(train_data[0])

{'question': ['12시', '땡'],
 'answer': ['하루가', '또', '가네요'],
 'label': ['하루가', '또', '가네요']}

In [25]:
vocab_data=TabularDataset('vocab_word.csv',format='csv',fields=[('vocab_word',V)],skip_header=True)

### 사전 정의
- 3개의 Field는 하나의 사전을 공유
- vocab_word라는 Q와A의 모든 문장을 합친 데이터로부터 단어사전을 만든 다음에
- 해당 단어사전을 Q,A,LABEL에 전달할 예정

In [26]:
V.build_vocab(vocab_data,min_freq=2)

Q.vocab=V.vocab
A.vocab=V.vocab
LABEL.vocab=V.vocab

In [ ]:
Q.vocab.stoi

In [ ]:
Q.vocab.itos[943]

'안녕'

In [ ]:
Q.vocab.stoi['안녕']

943

### 시드 고정 후 데이터 분리

In [27]:
import random
import torch
SEED=123
random.seed(SEED)
torch.manual_seed(SEED)

train_data,val_data=train_data.split(split_ratio=0.9,random_state=random.seed(SEED))

### 데이터 로더 생성

In [28]:
from torchtext.data import Iterator

In [29]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

In [ ]:
batch=next(iter(train_iter))
batch.question

tensor([[ 493,    0,    0, 1647,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [  24,    0,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [ ]:
batch.answer

tensor([[   2, 9027, 7444,    4,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2, 7855, 5459, 2818, 7854, 4065,   15, 2818,  128,    3,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [ ]:
batch.label

tensor([[9027, 7444,    4,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [7855, 5459, 2818, 7854, 4065,   15, 2818,  128,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [ ]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

# 모델 구현

In [30]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256
device='cuda' if torch.cuda.is_available() else 'cpu'
vocab_size=len(V.vocab)

In [31]:
## 모듈 불러오기
import torch
import torch.nn as nn

In [32]:
class Encoder(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Encoder,self).__init__()
    self.embedding=nn.Embedding(vocab_size,emb_dim)
    self.gru=nn.GRU(emb_dim,hidden_dim,batch_first=True)
    self.dense=nn.Linear(hidden_dim,dense_dim)

  def forward(self,x):
    x=self.embedding(x)  # (batch_size, Max_len, emb_dim)
    outputs,last_hidden_state=self.gru(x) # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( 1, batch_size, hidden_dim )
    return self.dense(outputs),self.dense(last_hidden_state), last_hidden_state #( batch_size, Max_len, dense_dim ) , ( 1, batch_size, hidden_dim ) , ( 1, batch_size, hidden_dim )

In [33]:
class Attention(nn.Module):
  def __init__(self,dense_dim):
    super(Attention,self).__init__()
    self.dense=nn.Linear(dense_dim,1)
    self.softmax=nn.Softmax(dim=1)

  def forward(self,outputs,last_hidden_state):
    # outputs: ( bacth_Size, 1, hidden_dim)
    # last_hidden_state : ( 1, bacth_Size, hidden_dim) or ( batch_size, 1, hidden_dim )
    last_hidden_state=last_hidden_state.view(BATCH_SIZE,1,UNIT) # ( bacth_size, 1, hidden_dim)
    after_sum=outputs+last_hidden_state  # ( batch_size, Max_len, dense_dim ) + ( bacth_Size, 1, hidden_dim) => broadcasting
    after_dense=self.dense(after_sum) # ( batch_size, Max_len, 1 )
    after_softmax=self.softmax(after_dense) # ( batch_size, Max_len, 1 )
    context_vector=torch.sum(after_softmax*outputs,dim=1) # ( batch_size, dense_dim )
    

    return torch.unsqueeze(context_vector,dim=1) # ( batch_size, 1, dense_dim )

In [6]:
a=torch.Tensor([[[1,2,3],[4,5,6]]])
print(a.size())
print(a.permute(1,0,2))
print(a.view(2,1,3))

torch.Size([1, 2, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


In [ ]:
a=torch.Tensor([[1,2,3],[4,5,6]])
print(a.size())
a=a.unsqueeze(1)
print(a.size())
print(a.expand(2,4,3)) # 크기가 1인 차원에 대해서만 가능

torch.Size([2, 3])
torch.Size([2, 1, 3])
tensor([[[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[4., 5., 6.],
         [4., 5., 6.],
         [4., 5., 6.],
         [4., 5., 6.]]])


In [ ]:
## Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1,2,3]]])
print(a.size())
print(b.size())
a+b

torch.Size([1, 2, 3])
torch.Size([1, 1, 3])


tensor([[[2., 4., 6.],
         [5., 7., 9.]]])

In [ ]:
## Multi token Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1,2,3]],[[4,5,6]]])
print(a.size())
print(b.size())
b+a

torch.Size([1, 2, 3])
torch.Size([2, 1, 3])


tensor([[[ 2.,  4.,  6.],
         [ 5.,  7.,  9.]],

        [[ 5.,  7.,  9.],
         [ 8., 10., 12.]]])

In [ ]:
## Softmax
softmax=nn.Softmax(dim=1)
a=torch.Tensor([[[3],[5],[4],[1]]])
out=softmax(a)
out

tensor([[[0.0889],
         [0.6572],
         [0.2418],
         [0.0120]]])

In [ ]:
## Weighted Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1],[2]]])
print(a.size())
print(b.size())
a*b

torch.Size([1, 2, 3])
torch.Size([1, 2, 1])


tensor([[[ 1.,  2.,  3.],
         [ 8., 10., 12.]]])

In [34]:
class Decoder(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Decoder,self).__init__()
    self.embedding=nn.Embedding(vocab_size,emb_dim)
    self.gru=nn.GRU(emb_dim+dense_dim,hidden_dim,batch_first=True)
    self.dense1=nn.Linear(hidden_dim,vocab_size)
    self.dense2=nn.Linear(hidden_dim,dense_dim)
  
  def forward(self,x,context,hidden):
    # x : ( batch_size, 1 )    
    # context : ( bath_size, 1, dense_dim==emb_dim)
    # hidden : ( 1, batch_size, hidden_dim)
    after_emb=self.embedding(x) # ( batch_size, 1 , emb_dim)
    after_cat=torch.cat([context,after_emb],dim=-1)  # ( batch_size, 1 , emb_dim+dense_dim)
    output,pre_hidden=self.gru(after_cat,hidden) # ( batch_size, 1 , hidden_dim), ( 1, batch_size, hidden_dim ) # hidden 에서는 batch_first가 적용되지 않는것을 주의!  
    predict_token=self.dense1(output) # ( batch_size, 1, hidden_dim) -> ( batch_size, 1, vocab_size )
    return  predict_token, self.dense2(pre_hidden), pre_hidden # (batch_size, 1, vocab_size) , (1, batch_size, hideen_dim) 다음 context 계산시 사용 , (1, batch_size, hideen_dim) 다음 gru pre_hidden으로 사용 

In [ ]:
a=torch.Tensor([[[1,2,3],[4,5,6]],[[3,2,1],[6,5,4]]])
b=torch.Tensor([[[1,2,3],[4,5,6]],[[4,5,6],[7,8,9]]])
print(a.size())
print(b.size())
torch.cat([b,a],dim=-1)

torch.Size([2, 2, 3])
torch.Size([2, 2, 3])


tensor([[[1., 2., 3., 1., 2., 3.],
         [4., 5., 6., 4., 5., 6.]],

        [[4., 5., 6., 3., 2., 1.],
         [7., 8., 9., 6., 5., 4.]]])

In [ ]:
a=torch.Tensor([[[1,2,3]],[[6,5,4]]])
b=torch.Tensor([[[1,2,3]],[[4,5,6]]])
print(a.size(),a,sep='\n')
print(b.size(),b,sep='\n')
torch.cat([a,b],dim=1)

torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[6., 5., 4.]]])
torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


tensor([[[1., 2., 3.],
         [1., 2., 3.]],

        [[6., 5., 4.],
         [4., 5., 6.]]])

In [53]:
dec_output=torch.LongTensor([0]).repeat(BATCH_SIZE,1,vocab_size)
dec_output.size()

torch.Size([2, 1, 9470])

In [35]:
class Seq2seq(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Seq2seq,self).__init__()
    self.encoder=Encoder(vocab_size,emb_dim,hidden_dim,dense_dim)
    self.attention=Attention(dense_dim)
    self.decoder=Decoder(vocab_size,emb_dim,hidden_dim,dense_dim)

  def forward(self,enc_input,dec_input): # enc_input : ( batch_size, max_len ) , dec_input : ( batch_size, max_ len )
    # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( batch_size, hidden_dim ), pre_hidden : (1, batch_size, hidden_dim)
    outputs,last_hidden_state,pre_hidden=self.encoder(enc_input) 
    context=self.attention(outputs,last_hidden_state) # ( batch_size, 1, dense_dim )

    dec_output=torch.zeros(BATCH_SIZE,1,vocab_size).to(device) # (batch_size, 1, vocab_size) 

    for i in range(0,MAX_LEN):
      predict,state,pre_hidden=self.decoder(dec_input[:,i].view(2,1),context,pre_hidden)  # (batch_size, 1, vocab_size) , ( 1, batch_size, hidden_dim ), ( 1, batch_size, hidden_dim )
      dec_output=torch.cat([dec_output,predict],dim=1) # 2차원 방향으로 서로 쌓음 (batch_size, i+2, vocab_size)
      context=self.attention(outputs,state) # ( batch_size, 1, dense_dim ) <= 다음 loop에서 context로 사용

    
    dec_output=dec_output[:,1:,:].transpose(1,2)  # (batch_size, max_len , vocab_size) axis=1의 맨 처음은 0으로 정의된 값이므로 제거 
    # 이렇게 돌고 맨 마지막으로 나온 25개의 연결된 값을 반환
    return dec_output # ( batch_size, vocab_size, max_len==25 )
    # -> 이에 대한 label : ( batch_size, max_len ) : crossenctorpyloss 사용하여 각각의 label에 해당하는 index(해당 단어의 index)의 요소의 값 가장 크도록 학습

  def inference(self,enc_input,max_len_answer):
    outputs,last_hidden_state,pre_hidden=self.encoder(enc_input)
    context=self.attention(outputs,last_hidden_state) # ( batch_size, 1, dense_dim )

    dec_input=torch.LongTensor([V.vocab.stoi['<sos>']]).to(device) # <sos> token의 vocab내 index : 2
    dec_input=dec_input.repeat(2,1) # ( batch_size, 1 ) 

    for i in range(0,max_len_answer):
      predict,state,pre_hidden=self.decoder(dec_input[:,i].view(2,1),context,pre_hidden)  # (batch_size, 1, vocab_size) , ( batch_size, hidden_dim )
      predict=predict.max(2)[1] # (batch_size, 1) # vocab_size에 해당하는 차원의 벡터 중에서 가장 값이 큰 원소의 index를 가져옴 
      if predict[0,-1]==V.vocab.stoi['<eos>']:
        break 
      dec_input=torch.cat([dec_input,predict],dim=-1) # 1차원 방향으로 서로 이어붙임 (batch_size,i+1)

      context=self.attention(outputs,state) # ( batch_size, 1, dense_dim ) <= 다음 loop에서 context로 사용
    
    # <eos> 전까지의 해당하는 단어 사전 내 index로 구성된 답변 문장 반환 
    return dec_input[0] # (answer_len,) batch내의 첫번째 sequence 반환 

In [10]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a[:,1].view(2,1)


tensor([[2.],
        [5.]])

In [ ]:
[V.vocab.stoi['<sos>']]

[2]

In [ ]:
V.vocab.stoi['<eos>']

3

In [ ]:
a=torch.Tensor([[[1,2,3,4],[5,6,7,8]],[[4,3,2,1],[9,8,7,6]]])
c=a.max(2)[1]
print(c)
print(c[0,-1])
print(c[0])
b=torch.Tensor([[1,3],[0,1]])
(a.max(2)[1]==b).sum()


tensor([[3, 3],
        [0, 0]])
tensor(3)
tensor([3, 3])


tensor(2)

In [ ]:
a=torch.Tensor([2])
a=a.repeat(2,1)
print(a)
torch.cat([a,a],dim=-1)

tensor([[2.],
        [2.]])


tensor([[2., 2.],
        [2., 2.]])

In [ ]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a=torch.unsqueeze(a,dim=1)
print(a.size(),a,sep='\n')

torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


In [ ]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a.size()
a[:,:2]

tensor([[1., 2.],
        [4., 5.]])

### 모델 컴파일

In [36]:
BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256
lr=1e-3
device='cuda' if torch.cuda.is_available() else 'cpu'
vocab_size=len(V.vocab)
vocab_size

9470

In [37]:
model=Seq2seq(vocab_size,EMB_DIM,UNIT,UNIT).to(device)
optimizer=torch.optim.Adam(model.parameters(),lr)
criterion=nn.CrossEntropyLoss(reduction='none')

In [ ]:
predict=Seq2seq(vocab_size,EMB_DIM,UNIT,UNIT).to(device)
predict=predict(batch.question.to(device),batch.answer.to(device))
loss=criterion(predict,batch.label.to(device))

In [ ]:
mask=torch.ones(loss.size()).to(device)
mask=torch.logical_not(mask==batch.label.to(device))
print(mask)
loss=(mask*loss).sum()
loss

In [ ]:
a=torch.Tensor([[3,5,6,2,1,1,1,1],[6,3,7,8,2,1,1,1]]) # label
b=torch.ones(a.size()) 
mask=torch.logical_not(a==b)
print(mask,int(mask.sum()))

tensor([[ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False]]) 9


In [45]:
a=torch.Tensor([[[1,2,3],[4,5,6]],[[7,8,9],[3,2,1]]])
print(a)
print(a.size())
print(a.mean())
a=a.transpose(1,2)
print(a)
print(a.size())

tensor([[[1., 2., 3.],
         [4., 5., 6.]],

        [[7., 8., 9.],
         [3., 2., 1.]]])
torch.Size([2, 2, 3])
tensor(4.2500)
tensor([[[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[7., 3.],
         [8., 2.],
         [9., 1.]]])
torch.Size([2, 3, 2])


## 모델 학습

In [38]:
from tqdm import tqdm

In [46]:
def loss_padding(loss,label):
  # 패딩을 loss 계산에서 지울 것이다. => label에서 index가 1인 부분에 대한 mask를 생성해서 곱해주자 / 추가적으로 accuracy 계산에서도 사용할 mask를 반환
    mask=torch.ones(loss.size()).to(device)
    mask=torch.logical_not(mask==label)
    loss=(mask*loss).mean()

    return loss,mask

In [47]:
def train(model,optimizer,criterion,train_iter):
  model.train()

  for batch in tqdm(train_iter):
    if batch.question.size()[0]!=2:
      break
    enc_input=batch.question.to(device)
    dec_input=batch.answer.to(device)
    label=batch.label.to(device)

    logit=model(enc_input,dec_input)
    loss,_=loss_padding(criterion(logit,label),label)  ## ( batch_size, max_len )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [48]:
def acc_padding(logit,label,mask):
  # padding을 포함gk지 않은 맞춘 개수를 반환 및 acc,loss 계산에 포함되는 전체 token 개수 return
  acc=(logit.max(2)[1].view(label.size())==label)
  acc=mask*acc # masking 
  acc=acc.sum()

  size=mask.sum() # 계산에 포함되는 batch내의 token의 개수
  
  return acc,size

In [49]:
def evaluation(model,optimizer,criterion,val_iter):
  model.eval()

  total_loss,acc,size=0,0,0
  for batch in tqdm(val_iter):
    if batch.question.size()[0]!=2:
      break
    enc_input=batch.question.to(device)
    dec_input=batch.answer.to(device)
    label=batch.label.to(device)


    logit=model(enc_input,dec_input)
    loss,mask=loss_padding(criterion(logit,label),label)  # logit : (batch_size, vocab_size, max_len) , label : (batch_size, max_len)
    total_loss+=loss


    logit=logit.transpose(1,2) # logit : (batch_size, max_len, vocab_size)
    acc,size=acc_padding(logit,label,mask)
    size+=size

  avg_loss=total_loss/size
  avg_acc=acc/size

  return avg_loss,avg_acc

In [ ]:
len(val_iter.dataset)

1182

In [ ]:
a=torch.Tensor([[[1,2,3,4],[5,6,7,8]],[[4,3,2,1],[9,8,7,6]]])
c=a.max(2)[1]
print(c)
b=torch.Tensor([[1,3],[0,1]])
(a.max(2)[1]==b).sum()

tensor([[3, 3],
        [0, 0]])


tensor(2)

In [50]:
import os

In [51]:
best_val_loss=None

for e in range(1,EPOCH+1):
  train(model,optimizer,criterion,train_iter)
  val_loss,val_acc=evaluation(model,optimizer,criterion,val_iter)

  print("Epoch : {} | val_loss : {:5.2f} , val_accuracy : {:5.2f}".format(e,val_loss.item(),val_acc.item()))

  if not best_val_loss or best_val_loss>val_loss:
    os.makedirs('best_weight',exist_ok=True)
    torch.save(model.state_dict(),'./best_weight/best_performance_weight.pt')
    best_val_loss=val_loss

  0%|          | 12/5321 [00:37<4:38:38,  3.15s/it]


KeyboardInterrupt: ignored

# 챗봇 사용하기

In [ ]:
import re

In [ ]:
# 실제 사용
# model output ( batch_size, vocab_size, max_len==25 )

model.load_state_dict(torch.load('./best_weight/best_performance_weight.pt'))

def chatbot(sentence):
  input=re.sub('[^ a-zA-Z0-9가-힣]',"",sentence).split()

  if len(input)<MAX_LEN:
    input=input+[V.vocab.stoi['<pad>']]*(MAX_LEN-len(input)) # padding
  else:
    input=input[:MAX_LEN] # truncate

  input=torch.LongTensor([V.vocab.stoi[word] for word in input]).to(device) # sentence to index
  enc_input=input.repeat(BATCH_SIZE,1) # ( batch_size, max_len ) # 모델에 사용하기 위해 사이즈에 맞게 준비, 이후 첫번째 샘플에 대한 예측값만 사용할 예정

  predict=model.inference(enc_input,14)

  return " ".join([V.vocab.itos[word] for word in predict][1:]) # <sos> 제거

In [ ]:
a=torch.Tensor([0,0,0])
b=torch.Tensor([1,3,4])
print(a+b)
a.repeat(2,1)

tensor([1., 3., 4.])


tensor([[0., 0., 0.],
        [0., 0., 0.]])

## 테스트

In [ ]:
sentence=input()
chatbot(sentence)

나는 사람이야


'<unk>'